In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import os

In [2]:
from ingest import load_split_file

In [3]:
unitsTab=pd.read_csv('codes/units.csv')
unitsTab.index = unitsTab['Unit Code']
unitsDict=unitsTab.to_dict()["Units"]

In [4]:
paramsTab=pd.read_csv('codes/parameters.csv')
paramsTab.index = paramsTab['Parameter Code']
paramsDict = paramsTab.to_dict()['Parameter']

In [5]:
load_split_file('raw_data/')

IsADirectoryError: [Errno 21] Is a directory: 'raw_data/'

In [68]:
def loadData(path):
    df=pd.read_csv(load_split_file(path)['RD'], delimiter='|')

    # drop all 'Qualifier' columns
    bad_columns = [c for c in df.columns if c.startswith('Qualifier')]
    bad_columns += ['Null Data Code', 'Sampling Frequency', 'Uncertainty', 'Monitor Protocol (MP) ID', 'Alternate Method Detectable Limit']
    df.drop(bad_columns, axis=1, inplace=True)

    # grab only rows which begin with 'RD'
#     df = df[df['# RD'] == 'RD']
#     df.dropna(subset=['Date'], inplace=True)

    # calculate datetimes and sort based on them

    df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Start Time'])
    df.index = df.datetime
    df.sort_index(inplace=True)
    df.drop(['Date', 'Start Time'], axis=1, inplace=True)

    # replace unit and parameter codes with readable data
#     df.Unit = df.Unit.astype(int)
#     df.Parameter = df.Parameter.astype(int)
    df.replace({'Unit':unitsDict, 'Parameter':paramsDict}, inplace=True)

    return df

In [69]:
basePath='raw_data/'
paths=[basePath + f for f in os.listdir(basePath) if f.endswith('.txt')]

In [70]:
df1 = loadData(paths[0])
df1.head(3)

,RD,Action Code,State Code,County Code,Site ID,Parameter,POC,Sample Duration,Unit,Method,Sample Value,datetime
datetime,,,,,,,,,,,,
2014-01-01 00:00:00,RD,I,42,133,8,Carbon monoxide,1,1,Parts per million,93,0.1,2014-01-01 00:00:00
2014-01-01 01:00:00,RD,I,42,133,8,Carbon monoxide,1,1,Parts per million,93,0.1,2014-01-01 01:00:00
2014-01-01 02:00:00,RD,I,42,133,8,Carbon monoxide,1,1,Parts per million,93,0.2,2014-01-01 02:00:00


In [64]:
df1.describe()

,State Code,County Code,Site ID,POC,Sample Duration,Method,Sample Value
count,26304.0,26304.0,26304.0,26304.0,26304.0,26304.0,24727.000000
mean,42.0,133.0,8.0,1.0,1.0,93.0,0.112108
std,0.0,0.0,0.0,0.0,0.0,0.0,0.161054
min,42.0,133.0,8.0,1.0,1.0,93.0,0.000000
25%,42.0,133.0,8.0,1.0,1.0,93.0,0.000000
50%,42.0,133.0,8.0,1.0,1.0,93.0,0.000000
75%,42.0,133.0,8.0,1.0,1.0,93.0,0.200000
max,42.0,133.0,8.0,1.0,1.0,93.0,1.600000


In [67]:
print(df1.Parameter.unique())

['Carbon monoxide']


In [73]:
for p in paths:
    print(p)
    df = loadData(p)
    print(p, '\t', df.index.min(), df.index.max(), df.Parameter.unique(), df.Unit.unique())

raw_data/Harrisburg_Lancaster_York_MSA_CO_2014-2016.txt
raw_data/Harrisburg_Lancaster_York_MSA_CO_2014-2016.txt 	 2014-01-01 00:00:00 2016-12-31 23:00:00 ['Carbon monoxide'] ['Parts per million']
raw_data/Harrisburg_Lancaster_York_MSA_PM10_2014-2016.txt
raw_data/Harrisburg_Lancaster_York_MSA_PM10_2014-2016.txt 	 2014-01-01 00:00:00 2016-12-31 23:00:00 ['PM10 Total 0-10um STP'] ['Micrograms/cubic meter (25 C)']
raw_data/Harrisburg_Lancaster_York_MSA_PM25SiteLevel_2014-2016.txt


KeyError: 'RD'

In [ ]:
filesFrame=pd.DataFrame(files, columns=['filename'])
filesFrame

In [ ]:
filesFrame['table'] = [loadData(basePath+f) for f in filesFrame.filename]

In [ ]:
filesFrame

In [ ]:
allData=pd.concat(list(filesFrame.table))
allData.sort_index(inplace=True)
allData.drop_duplicates(['datetime','value','parameter','unit'], inplace=True)
allData.tail(10)

In [ ]:
print(allData.parameter.unique())
[CO, O3, PM2, WS, Temp]=allData.parameter.unique()

In [ ]:
allData[allData.parameter==CO].value.plot()
# allData[allData.parameter==O3].value.plot()

In [ ]:
allData[allData.parameter==O3].value.plot()

In [ ]:
allData[allData.parameter==PM2].value.plot()

In [ ]:
allData[allData.parameter==WS].value.plot()

In [ ]:
allData[allData.parameter==Temp].value.plot()